In [1]:
# with all production modes
# make corrections to signal yield for the simulation modelling uncertainties


import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections
import os

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
sys.path.append('/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/python/')
from helper import make_datacard, make_datacard_2sig, weight_calc
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0

wH = 1
Z_MASS = 91.2


# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.22/02
3.6.8 (default, Apr  2 2020, 13:34:55) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [34]:
fpath =OrderedDict()
tree = OrderedDict()
mass = [15, 40, 55]

prod = ['ggH', 'VBFH_H','ZH', 'WH', 'ttH_H','ggZH']
decay = 'dddd'


# prod = ['ggH','VBFH','ZH', 'WH', 'ttH_H','ggZH']
# decay = '4Tau'


# decay = 'bbbb'

category = 0

mass = [15, 40, 55]
if not decay == 'bbbb': mass = [7, 15, 40, 55]

OLD_CTAU = np.array([100, 1000, 10000, 100000])#in mm


years = ['Summer16', 'Fall17', 'Fall18']
lumi = {
    'Summer16': 35.92 * 1000,
    'Fall17': 41.53 * 1000,
    'Fall18': 59.74 * 1000,
}
ntupler_version = 'V1p17/'

data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/Data2018/v5/v100/normalized/'

fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root'
mc_path = {}
analyzer_version = 'v1/v66/'
analyzer_version = 'v1/v82/'
analyzer_version = 'v1/v86/'




analyzer_version = 'v1/v100/'
analyzer_version = 'v1/v106/'
# analyzer_version = 'v1/v111/'


# analyzer_version = 'v1/v108/'



mc_path['ggH'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
mc_path['VBFH'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'


analyzer_version = 'v2/v100/'
analyzer_version = 'v2/v106/'


# if decay == 'bbbb': analyzer_version = 'v2/v100/'


mc_path['ZH'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'
mc_path['WH'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'
mc_path['WminusH'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'

mc_path['WplusH'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'

mc_path['ttH_H'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'
mc_path['VBFH_H'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'
mc_path['ggZH'] = mc_path['ZH']



# analyzer_version = 'v2/v77/'
# analyzer_version = 'v2/v95/'



# analyzer_version = 'v2/v95/'




for m in mass:
    for ct in OLD_CTAU:
        for p in prod:
            if p == 'others':continue
            if p == 'VBFH' and decay == 'bbbb':
                key = 'MC_'+p+'_'+str(m)+'_'+str(ct)                       
                fpath[key] = mc_path[p]+p+'_HToSSTo4b_MH-125_MS-'+str(m)+'_ctau-'+str(ct)+'_137000pb_weighted.root'
            elif p == 'ggH' or p == 'VBFH':
                key = 'MC_'+p+'_'+str(m)+'_'+str(ct)                       
#                 fpath[key] = mc_path[p]+p+'_HToSSTo'+decay+'_MH-125_MS-'+str(m)+'_ctau-'+str(ct)+'_TuneCP5_13TeV-powheg-pythia8_41530pb_weighted.root'
                fpath[key] = mc_path[p]+p+'_HToSSTo'+decay+'_MH-125_MS-'+str(m)+'_ctau-'+str(ct)+'_137000pb_weighted.root'
                
#             elif p == 'ggZH':
#                 key = 'MC_'+p+'_'+str(m)+'_'+str(ct)  
#                 fpath[key] = mc_path[p]+'ggZHToSS_SToBB_ms40_pl1000_137000pb_weighted.root'
# #                 fpath[key+'LL'] = mc_path[p]+'ggZHToSS_SToBB_ZToLL_ms40_pl1000_137000pb_weighted.root'
# #                 fpath[key] = mc_path[p]+'ggZHToSS_SToBB_ZToQQ_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
# #                 fpath[key+'NuNu'] = mc_path[p]+'ggZHToSS_SToBB_ZToNuNu_ms40_pl1000_137000pb_weighted.root'
                
            elif p == 'VBFH_H':
                key = 'MC_'+p+'_'+str(m)+'_'+str(ct)  
                fpath[key] = mc_path[p]+'VBFHToSS_STodd_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
            else:
                key = 'MC_'+p+'_'+str(m)+'_'+str(ct)                       
                if decay == 'bbbb':fpath[key] = mc_path[p] + p+'ToSS_SToBB_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
                if decay == 'dddd':fpath[key] = mc_path[p] + p+'ToSS_STodd_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
                if decay == '4Tau':fpath[key] = mc_path[p] + p+'ToSS_SToTauTau_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
                if p == 'ggZH':
                    if decay == 'bbbb':fpath[key] = mc_path[p] + 'ZHToSS_SToBB_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
                    if decay == 'dddd':fpath[key] = mc_path[p] + 'ZHToSS_STodd_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'
                    if decay == '4Tau':fpath[key] = mc_path[p] + 'ZHToSS_SToTauTau_ms'+str(m)+'_pl'+str(ct)+'_137000pb_weighted.root'


NEvents = {}
NEvents_genweight = {}
for k,v in fpath.items():
#     print(k,v)
#     if 'data' in k:continue
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    
    w = tree[k]["weight"].array()
    if not 'data' in k: 
        print(k, root_dir['NEvents']._fEntries)
#         print(k,np.mean(w))
# v = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/HiggsPtWeights/ttHToggZH_HiggsPtReweight.root'
v = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/HiggsPtWeights/ZHToggZH_HiggsPtReweight.root'


root_dir = uproot.open(v) 
h_reweight = root_dir['higgsPthiggsEta']

MC_ggH_7_100 6623999.0
MC_VBFH_H_7_100 1495600.0
MC_ZH_7_100 382100.0
MC_WH_7_100 998100.0
MC_ttH_H_7_100 499700.0
MC_ggZH_7_100 382100.0
MC_ggH_7_1000 6796000.0
MC_VBFH_H_7_1000 134000.0
MC_ZH_7_1000 416700.0
MC_WH_7_1000 394500.0
MC_ttH_H_7_1000 198600.0
MC_ggZH_7_1000 416700.0
MC_ggH_7_10000 6922000.0
MC_VBFH_H_7_10000 148400.0
MC_ZH_7_10000 446700.0
MC_WH_7_10000 395300.0
MC_ttH_H_7_10000 195600.0
MC_ggZH_7_10000 446700.0
MC_ggH_7_100000 6958000.0
MC_VBFH_H_7_100000 1495000.0
MC_ZH_7_100000 392199.0
MC_WH_7_100000 998399.0
MC_ttH_H_7_100000 499600.0
MC_ggZH_7_100000 392199.0
MC_ggH_15_100 6638000.0
MC_VBFH_H_15_100 1484200.0
MC_ZH_15_100 321900.0
MC_WH_15_100 997296.0
MC_ttH_H_15_100 498100.0
MC_ggZH_15_100 321900.0
MC_ggH_15_1000 6660998.0
MC_VBFH_H_15_1000 150000.0
MC_ZH_15_1000 422400.0
MC_WH_15_1000 396098.0
MC_ttH_H_15_1000 198800.0
MC_ggZH_15_1000 422400.0
MC_ggH_15_10000 6985317.0
MC_VBFH_H_15_10000 150000.0
MC_ZH_15_10000 422700.0
MC_WH_15_10000 387498.0
MC_ttH_H_15_10000 1

# load bdt

In [35]:
# make predictions for test data

import pickle
# model = pickle.load(open( 'bdt_flatten_noEvtSelection.pickle.dat', "rb" ))
# bdt_name = 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_XYspread_v12'
bdt_name = 'bdt_flatten_metfilter_me1112nohits_eta2p0_testsize0p2_noSpread_v2'


# model = pickle.load(open( bdt_name+'.pickle', "rb" ))


# nCsc with different hit vetoing

In [36]:

JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False
ring_cut = 50
cut_based = True
cut_based_version = 'v4'

intime = True
DPHI_CUT = 1
weight = {}
weight_event = {}
weight_ctau = {}
lumiSec = {}
evtNum = {}
nCsc = {}
npv = {}
npu = {}
runNum = {}
nCsc_JetMuonVetoCluster0p4_Me1112Veto = {}
gLLP_csc = {}
cscClusterSize = {}
cscClusterTime = {}
nCscClusters = {}
selections_cluster = {}
sel_cluster = {}
sel_jetveto = {}
met_trigger = {}
met = {}
gLLP_beta = {}
jetPt = {}
jetPhi = {}
metPhi = {}
angle ={}
nLeptons = {}

pileupWeight = {}
gLLP_ctau = {}
npv = {}
nRechitClusters = {}
nJets = {}
nJets_50gev = {}
cscRechitClusterTimeDiff = {}
cscRechitCluster_match_gLLP = {}
higgsPtWeight = {}
cscRechitClusterXSpread = {}
cscRechitClusterYSpread = {}
cscRechitClusterXYSpread = {}
cscRechitClusterNStation = {}
cscRechitClusterAvgStation = {}
cscRechitClusterEtaPhiSpread = {}
cscRechitClusterPhiSpread = {}
cscRechitClusterEtaSpread = {}
cscRechitClusterX = {}
cscRechitClusterY = {}
cscRechitClusterZ = {}
cscRechitClusterPhi = {}
cscClusterJetVetoPt = {}
cscRechitClusterEta = {}
cscRechitClusterZSpread = {}
cscRechitClusterNStation10  = {}
cscRechitClusterAvgStation10 = {}
cscRechitClusterMaxStationRatio = {}
cscRechitClusterNChamber = {}
cscRechitClusterMet_dPhi = {}
jetMet_dPhiMin30 = {}
jetMet_dPhiMin = {}
dphiMet_cluster = {}
nRechits_sr = {}
jetMet_dPhiMin30_sr = {}
nCscRings = {}
nDtRings = {}
bdt_score = {}
a = {}
b = {}
c = {}
d = {}
sel_ev = {}
bdt_sel = {}
sf_facScaleUp = {}
sf_facScaleDown = {}
sf_renScaleUp = {}
sf_renScaleDown = {}
sf_facRenScaleUp = {}
sf_facRenScaleDown = {}
metSF = {}
legend = {}
runNum = {}
lumiSec = {}
evtNum = {}
ggZH_weight = {}
higgsEta = {}
higgsPt = {}
gLLP_other_r = {}
gLLP_other_z = {}
cscRechitCluster3_match_gLLP_decay_r = {}
cscRechitCluster3_match_gLLP_decay_z = {}
cscRechitCluster3_match_gLLP_csc = {}

index = {}
legend['data_oot'] = 'Data OOT region'
legend['data_intime'] = 'Data in-time region'
legend['mc_signal'] = 'signal MC in time'
legend['mc_intime'] = 'MC in-time background'
legend['mc_oot'] = 'MC OOT background'
legend['mc_bkg'] = 'QCD 50toInf background'

# keys = ['data','mc_intime_bkg','mc_oot_bkg','mc_signal']
keys = ['data_intime','data_oot','mc_signal','mc_bkg']
keys = ['data_intime','mc_signal',]
keys = ['data_intime','mc_signal']
keys = ['data_oot_vr','data_oot_sr', 'data_intime_vr', 'data_intime_sr', 'mc_vr','mc_sr']
keys = ['data_oot_vr','data_oot_sr', 'data_intime_vr', 'data_intime_sr', 'mc100_vr','mc100_sr', 'mc1000_vr', 'mc1000_sr']
cluster_index = '3'
print("cut based", cut_based_version)



for k in list(tree.keys())+['data_oot_sr', 'data_oot_vr', 'data_intime_vr','data_intime_sr']:
#     if 'data' in k:continue
########### SELECTION: CLUSTERS ############
    if 'data' in k: T = tree['data']
    else: T = tree[k]

    sel_rechitcluster = np.abs(T.array('cscRechitCluster' + cluster_index + 'Eta')) < 2.0
    
    me1112_veto = 0
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus12') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus12') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_MB1Seg_0p4') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_RE12_0p4') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_RB1_0p4') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'TimeSpread') <= 20)

    if 'oot' in k:
        
        sel_rechitcluster = np.logical_and(sel_rechitcluster,  T.array('cscRechitCluster' + cluster_index + 'TimeTotal') < -12.5)
    else:
        sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeTotal') < 12.5, T.array('cscRechitCluster' + cluster_index + 'TimeTotal') > -5.0))
#         sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeTotal') < 12.0, T.array('cscRechitCluster' + cluster_index + 'TimeTotal') > -4.5))


    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') < JET_PT_CUT)
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt') < MUON_PT_CUT)


#     sel_rechitcluster = np.logical_and( sel_rechitcluster, np.logical_or(T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt') < 10,\
#                               T.array('cscRechitCluster' + cluster_index + 'MuonVetoGlobal') == 0))


    
    
#     if 'MC' in k:
#         sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_gLLP_csc'))



########### SELECTION: JETS ############
    
    sel_jet = np.logical_and(T.array('jetPt') > jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )

########### SELECTION: EVENTS ############

    sel_ev[k] = T.array('METNoMuTrigger')
#     sel_ev[k] = T.array('HLTDecision')[:,467]
#     sel_ev[k] = T.array('HLTDecision')[:,467]


#     sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metXYCorr') >= 200)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise') >= 200)
    sel_ev[k]  = np.logical_and(sel_ev[k], T.array('category') == category)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('nLeptons') == 0)
    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    sel_ev[k]  = np.logical_and(sel_ev[k],sel_rechitcluster.sum() >= 1)

#     sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('ZCategory') == 2)



    sel_ev[k] = np.logical_and(sel_ev[k], (T.array('nDtRings')+T.array('nCscRings'))<10)
    sel_ev[k] = np.logical_and(sel_ev[k],T.array('Flag2_all'))

    w = T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF')
    

#     if 'MC' in k:
#         sel_ev[k] = np.logical_and(sel_ev[k], np.sum(T.array('gLLP_csc'),axis = 1) > 0)

########### BRANCHES ############

    
    ##### bdt variables ####

    cscRechitClusterNStation10[k] = T.array('cscRechitCluster' + cluster_index + 'NStation10')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterEta[k] = T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterAvgStation10[k] = T.array('cscRechitCluster' + cluster_index + 'AvgStation10')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterPhi[k] = T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    if len(cscRechitClusterAvgStation10[k])>0:
        if cut_based:     
            if cut_based_version == 'v4':
                cond2 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==2, np.abs(cscRechitClusterEta[k]) < 1.6)
                cond3 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==3, np.abs(cscRechitClusterEta[k]) < 1.6)
                cond4 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==4, np.abs(cscRechitClusterEta[k]) < 1.8)
                cond1 = np.logical_and(cscRechitClusterNStation10[k]==1, np.logical_or(np.logical_or(np.abs(cscRechitClusterAvgStation10[k])==1, cond2), np.logical_or(cond3, cond4)))
                cond2 = np.logical_and(cscRechitClusterNStation10[k] > 1, np.abs(cscRechitClusterEta[k]) < 1.9)
                bdt_sel[k] = np.logical_or(np.logical_or(cond1, cond2), np.logical_or(cond3, cond4))
            else:
                print("CUT BASED ERROR")
            if 'vr' in k:
                bdt_sel[k] = np.logical_not(bdt_sel[k])
        else:
            print('bdt based')
            if k == 'data_oot_sr':
                bdt_sel[k] = bdt_score[k] >= BDT_CUT
            elif 'vr' in k:
                bdt_sel[k] = bdt_score[k] < BDT_CUT            
            elif k == 'data':
                bdt_sel[k] = bdt_score[k] < BDT_CUT 
            else:
                bdt_sel[k] = bdt_score[k] >= BDT_CUT
        print("effiency",np.count_nonzero(bdt_sel[k])/len(bdt_sel[k]))

        dphiMet_cluster[k] = np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()[bdt_sel[k]]


        jetMet_dPhiMin30[k] = T.array('jetMet_dPhiMin')[sel_ev[k]][bdt_sel[k]]
        if 'data' in k:metSF[k]= np.ones(dphiMet_cluster[k].shape, dtype=bool)
        else: metSF[k] = np.abs(T.array('metSF'))[sel_ev[k]][bdt_sel[k]]


        higgsPt[k] = T.array('gHiggsPt')[sel_ev[k]][bdt_sel[k]]
        higgsEta[k] = T.array('gHiggsEta')[sel_ev[k]][bdt_sel[k]]


#         ggZH_weight[k] = h_reweight.values[np.argmax(h_reweight.edges>higgsPt[k][:,None],axis=1)-1]
        
        ggZH_weight[k]=h_reweight.values[np.argmax(h_reweight.edges[0]>higgsPt[k][:,None],axis=1)-1, np.argmax(h_reweight.edges[1]>np.abs(higgsEta[k])[:,None],axis=1)-1]


        if 'ggH' in k: weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))[sel_ev[k]][bdt_sel[k]]
        else:weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev[k]][bdt_sel[k]]
        if 'ggZH' in k: weight[k] *= ggZH_weight[k]
        cscClusterSize[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_rechitcluster][sel_ev[k]][bdt_sel[k]]
        nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] = cscClusterSize[k][:,0]
        lumiSec[k] = T.array('lumiSec')[sel_ev[k]][bdt_sel[k]]
        runNum[k] = T.array('runNum')[sel_ev[k]][bdt_sel[k]]
        evtNum[k] = T.array('evtNum')[sel_ev[k]][bdt_sel[k]]
        metPhi[k] = T.array('metPhiEENoise')[sel_ev[k]][bdt_sel[k]]
        met[k] = T.array('metEENoise')[sel_ev[k]][bdt_sel[k]]



#         print(np.sum(weight[k]),np.sum(weight[k]*ggZH_weight[k]))

#         cscRechitCluster3_match_gLLP_decay_r[k] = np.abs(T.array('cscRechitCluster' + cluster_index + '_match_gLLP_decay_r'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()[bdt_sel[k]]
#         cscRechitCluster3_match_gLLP_decay_z[k] = np.abs(T.array('cscRechitCluster' + cluster_index + '_match_gLLP_decay_z'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()[bdt_sel[k]]
#         cscRechitCluster3_match_gLLP_csc[k] = np.abs(T.array('cscRechitCluster' + cluster_index + '_match_gLLP_csc'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()[bdt_sel[k]]
        
        
        
#         index[k] = np.abs(T.array('cscRechitCluster' + cluster_index + '_match_gLLP_index'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()[bdt_sel[k]]

#         gLLP_other_r[k] = T.array('gLLP_decay_vertex_r')[sel_ev[k]][bdt_sel[k]][list(range(np.count_nonzero(bdt_sel[k]))),[np.abs(1-index[k])]][0]
#         gLLP_other_z[k] = T.array('gLLP_decay_vertex_z')[sel_ev[k]][bdt_sel[k]][list(range(np.count_nonzero(bdt_sel[k]))),[np.abs(1-index[k])]][0]


        
    else:
        dphiMet_cluster[k] = np.abs(T.array('cscRechitCluster' + cluster_index + 'Met_dPhi'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()
        if 'ggH' in k:weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight'))[sel_ev[k]]
        else:weight[k] = (T.array('weight')*T.array('pileupWeight'))[sel_ev[k]]
        if 'data' in k:weight[k] = np.ones(weight[k].shape, dtype=bool)
        cscClusterSize[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_rechitcluster][sel_ev[k]]
        nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] = cscClusterSize[k][:,0]
        if 'data' in k:metSF[k]= np.ones(dphiMet_cluster[k].shape, dtype=bool)
        else: metSF[k] = np.abs(T.array('metSF'))[sel_ev[k]]
        lumiSec[k] = T.array('lumiSec')[sel_ev[k]]
        runNum[k] = T.array('runNum')[sel_ev[k]]
        evtNum[k] = T.array('evtNum')[sel_ev[k]]
        print("no BDT applied")

cut based v4
effiency 0.2985971943887776
effiency 0.8920758385999028
effiency 0.8892794376098418
effiency 0.8802660753880266
effiency 0.8664898320070734
effiency 0.8969823100936525
effiency 0.8802660753880266
effiency 0.9036431733060947
effiency 0.9102564102564102
effiency 0.9042735042735043
effiency 0.8978494623655914
effiency 0.9069020866773676
effiency 0.9042735042735043
effiency 0.8772563176895307
effiency 0.9111111111111111
effiency 0.8914728682170543
effiency 0.8869565217391304
effiency 0.9047619047619048
effiency 0.8914728682170543
effiency 0.9152542372881356
effiency 0.8589743589743589
effiency 0.8
effiency 0.6388888888888888
effiency 0.7450980392156863
effiency 0.8
effiency 0.8458980044345898
effiency 0.8685208596713021
effiency 0.8518518518518519
effiency 0.8249027237354085
effiency 0.8823529411764706
effiency 0.8518518518518519
effiency 0.9113124656781988
effiency 0.8918128654970761
effiency 0.9038929440389294
effiency 0.8906666666666667
effiency 0.9250302297460702
effiency 

# signal yield summary table

In [18]:
ctaus = [ '100', '1000','10000','100000' ]


BR = 0.01
N_RECHIT_CUT = 130
DPHI_CUT = 0.75
corrections = 0.90332048232


var = dphiMet_cluster
# DPHI_CUT = 0.6      
# var = jetMet_dPhiMin30
for N_RECHIT_CUT in np.arange(60,220,10):
    if not N_RECHIT_CUT==130:continue
    total_sig = 0
    for m in mass:
#         if not m == 55:continue
        signal_rate = []
        unc_rate = []
        signal_unc = []
        for ct in ctaus:   
#             if not ct == '1000':continue
            signal = 0
            sig_unc = 0
            shape_unc_temp = 0
            ctf = int(ct)
            if ctf < OLD_CTAU[0]:
                old_ctau_temp = np.array([OLD_CTAU[0]])
            else:
                for j, ct0 in enumerate(OLD_CTAU):
                    if ct0 == ctf: 
                        old_ctau_temp = np.array([int(ctf)])
                        break

                    elif ct0 > ctf:
                        old_ctau_temp = np.array([OLD_CTAU[j-1], OLD_CTAU[j]])
                        old_ctau_temp = np.array([OLD_CTAU[j]])
                        break
                    if j == len(OLD_CTAU)-1: 
                        old_ctau_temp = np.array([OLD_CTAU[j]])
#             print(ct,old_ctau_temp)

            weight_sum = 0
            weight_len = 0
            for j,ct0 in enumerate(old_ctau_temp):
#                 if VBFH: prods = ['ggH', 'VBFH']
#                 else: prods = ['ggH']
#                 prods = ['ggH', 'VBFH','WH','ZH','ttH_H','ggZH']
                prods = ['ggZH']
                for p in prods:
#                 for p in ['','LL','QQ','NuNu']:
#                     production = 'ttH'
                    
                    k = 'MC_'+p+'_'+str(m)+'_'+str(ct0)+''
                    

                    T = tree[k]
                    if np.count_nonzero(sel_ev[k]) == 0: continue
                    gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                    if len(gLLP_ctau) == 0: continue
                    weight_ctau = weight_calc(gLLP_ctau, float(ct)/10, old_ctau_temp/10) # convert everything to cm
#                     if p == 'ggZH':w = weight[k]*ggZH_weight[k]
#                     else:w = weight[k]
                    w = weight[k]
#                     print(ggZH_weight[k])
#                         w = weight_ctau* weight[k]
#                     shape_unc = signal_weight(unc_list, np.abs(cscRechitClusterEta[k][bdt_sel[k]]))
                    # higgs pt, signal uncertainty weight
                    cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, np.abs(var[k])<DPHI_CUT)#bin D
#                     cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, np.abs(var[k])>=DPHI_CUT) #bin A
# #                     cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, np.abs(var[k])>=DPHI_CUT) #bin B
#                     cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, np.abs(var[k])<DPHI_CUT) #bin C



                    signal += np.sum(w[cond])
                    sig_unc +=np.sum(w[cond]*w[cond])
#                     shape_unc_temp = (shape_unc_temp**2+np.sum(w[cond]*shape_unc[cond])**2)**0.5


            signal_rate.append(signal)
            signal_unc.append(sig_unc**0.5)
            unc_rate.append(shape_unc_temp)
        signal_rate = np.array(signal_rate)
        unc_rate = np.array(unc_rate)
        signal_unc = np.array(signal_unc)
#         print(N_RECHIT_CUT, '\t', signal_rate[0]*BR)
        print(m,'GeV &', ' & '.join(map(str, [round(num,1) for num in signal_rate*BR*corrections])),'\\\\')
#         print(m,'GeV &', ' & '.join(map(str, [round(num,1) for num in signal_rate*limit[m]])),'\\\\')

#         print(m, '\t', '\t'.join(map(str, [round(num,2) for num in signal_unc*BR])))
#         print(m, '\t', '\t'.join(map(str, [round(num,1) for num in signal_rate*BR*corrections])))
#         print(m, '\t', '\t'.join(map(str, [round(num,1) for num in signal_rate*BR])))


#         print(m, '\t', '\t'.join(map(str, [round(num,2) for num in signal_unc/signal_rate])))


#         print(m, '\t'.join(map(str,[round(num,2) for num in unc_rate*BR]))) #signal mc relative uncertainty
        total_sig += np.sum(signal_rate)
# print(total_sig*BR)

#     print(m, '\t'.join(map(str,unc_rate/signal_rate))) #signal mc relative uncertainty



15 GeV & 0.1 & 0.4 & 0.1 & 0.0 \\
40 GeV & 0.0 & 0.4 & 0.3 & 0.1 \\
55 GeV & 0.0 & 0.1 & 0.4 & 0.1 \\


In [6]:
limit = {
    15: [0.00543359, 0.00171431, 0.00841978, 0.08401191],
40: [0.23098513 ,0.00200566, 0.00241547, 0.01832026],
55: [1.00000000e+06 ,4.70192974e-03 ,1.82907116e-03 ,1.23568459e-02]
}



In [39]:
key = 'MC_ggH_40_1000'
condd = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[key]>=N_RECHIT_CUT, np.abs(var[key])<DPHI_CUT)
print(np.count_nonzero(cscRechitCluster3_match_gLLP_csc[key][condd])/len(cscRechitCluster3_match_gLLP_csc[key][condd]))
print(np.count_nonzero(condd))
cond = np.logical_not(cscRechitCluster3_match_gLLP_csc[key])
# print(cscRechitCluster3_match_gLLP_decay_r[key][cond])
# print(cscRechitCluster3_match_gLLP_decay_z[key][cond])

for i in range(len(cscRechitCluster3_match_gLLP_decay_r[key][cond])):
    r, z = abs(cscRechitCluster3_match_gLLP_decay_r[key][cond][i]), abs(cscRechitCluster3_match_gLLP_decay_z[key][cond][i])
    if nCsc_JetMuonVetoCluster0p4_Me1112Veto[key][cond][i]<130:continue
    if np.abs(dphiMet_cluster[key][cond][i])>0.75:continue
    if r<695.5 and z>400 and z<1100 and r>100:print(r,z)
    else: print(r, z,nCsc_JetMuonVetoCluster0p4_Me1112Veto[key][i], np.abs(var[key])[cond][i])
 

0.9552995391705069
2170
585.8888 601.1224
493.17136 549.8971
515.6612 541.5628
484.27054 555.4028
592.5819 577.2477
675.82153 657.04193
589.9776 580.93353
640.5917 639.31366
566.8548 580.5812
612.0582 607.73193
608.94275 608.76227
603.5485 589.61926
557.1205 562.9656
495.12888 533.5148
491.64462 530.70197
617.9657 619.7865
598.1483 602.8528
638.69745 638.0258
611.40845 609.5604
577.0126 570.03357
587.5955 596.3765
547.97614 549.7628
472.06573 557.2741
477.2299 493.95123
523.82605 555.1639
624.6749 639.15753
645.4383 648.5565
496.4647 490.87946
583.3065 595.1655
667.14746 657.56635
580.7768 586.41235
491.92392 544.80994
482.17548 496.75262
616.7016 609.3995
530.19476 559.9751
622.0559 622.3765
525.2476 554.47546
488.87192 534.378
613.5425 611.0761
474.1113 546.14343
596.8385 607.89307
596.4745 587.9145
584.9745 553.4688
616.7432 588.1833
605.95905 612.2923
535.2216 559.34076
611.59985 610.87366
629.7858 637.6442
648.95074 648.2171
574.7701 572.6918
491.63852 567.7304
626.1801 623.0141
6

# signal uncertainties

In [37]:
jetVeto = {}
muonVeto = {}
rechitVeto = {}
cut_based_eff_unc = {}
clusterEff_unc = {}
higgsPtWeight = {}
JES = {}
pileup = {}
# ctau_reweight = {}
mc_stats = {}
scaling = {}
lumi = {}
time_spread = {}
time = {}
theory = {}
pdf = {}
ggH_higgsPt , VBFH_higgsPt , WH_higgsPt , ZH_higgsPt , ttH_higgsPt , ggZH_higgsPt =({} for i in range(6)) 
ggH_qcdScale , VBFH_qcdScale , WH_qcdScale , ZH_qcdScale , ttH_qcdScale , ggZH_qcdScale =({} for i in range(6)) 
ggH_pdf , VBFH_pdf , WH_pdf , ZH_pdf , ttH_pdf , ggZH_pdf =({} for i in range(6)) 
Nrechit_weight_file = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/Uncertainties/Nrechit_weight.txt'

cut_based_unc = [0.0867, 0.0754]
clustering_unc = [0.1506, 0.1047]

for m in mass:
    for ct in OLD_CTAU:
        for p in prod:
            if p == 'others':k = 'MC_ggH_'+str(m)+'_'+str(ct)   
            else:k = 'MC_'+p+'_'+str(m)+'_'+str(ct)         
            if p == 'others': key = 'MC_others_'+str(m)+'_'+str(ct)   
            else: key = k
            cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=130, nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<250)
            Nrechits130= np.sum((weight[k])[cond])
            cond = nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=250
            Nrechits250= np.sum(weight[k][cond])
            if Nrechits130+Nrechits250 >0:
                Nrechits130_ratio = Nrechits130/(Nrechits130+Nrechits250)
                Nrechits250_ratio = Nrechits250/(Nrechits130+Nrechits250)
            else:
                Nrechits130_ratio = 0
                Nrechits250_ratio = 0

            lumi[key] = [0.018]*4
            jetVeto[key] = [0.0154]*4
#             muonVeto[key] = [0.071]*4
#             muonVeto[key] = [0.039]*4
#             muonVeto[key] = [0.0028]*4
            muonVeto[key] = [0.076*0.7]*4





            rechitVeto[key] = [0.031]*4
            time[key] = [0.009]*4
            time_spread[key] = [0.028]*4


            cutBased_highNhits = Nrechits130_ratio*cut_based_unc[0]+Nrechits250_ratio*cut_based_unc[1]
            cut_based_eff_unc[key]=[cutBased_highNhits, 0.0075, 0.0075, cutBased_highNhits]
            clusterEff_highNhits = Nrechits130_ratio*clustering_unc[0]+Nrechits250_ratio*clustering_unc[1]
#             clusterEff_unc[key]  = [clusterEff_highNhits, 0.0089, 0.0089, clusterEff_highNhits]
            clusterEff_unc[key]  = [0.011, 0.0089, 0.0089, 0.011] #shifting Nrechit cut from 130 to 135


            
            JES[key] = [0.084,0.083,0.042,0.041]
            pileup[key] = [0.008, 0.0110, 0.0110, 0.0110]
            


            ggH_higgsPt[key], VBFH_higgsPt[key], WH_higgsPt[key] , ZH_higgsPt[key] , ttH_higgsPt[key] , ggZH_higgsPt[key] = ([0.0]*8,)*6
            ggH_qcdScale[key], VBFH_qcdScale[key], WH_qcdScale[key] , ZH_qcdScale[key] , ttH_qcdScale[key] , ggZH_qcdScale[key] = ([0.0]*8,)*6
            ggH_pdf[key] , VBFH_pdf[key] , WH_pdf[key] , ZH_pdf[key] , ttH_pdf[key] , ggZH_pdf[key] = ([0.0]*4,)*6
            if 'ggH' in key: 
                ggH_higgsPt[key] = [0.205, 0.205,0.207,0.208, 0.133,0.133,0.134,0.134] #down*4/up*4
                ggH_qcdScale[key] = [0.067]*4+[0.046]*4 #down/up
                ggH_pdf[key] = [0.032]*4

            elif 'VBFH' in key: 
                VBFH_higgsPt[key] = [0.046, 0.159, 0.007, 0.006, 0.022, 0.032, 0.012, 0.010] #down/up
                VBFH_qcdScale[key] = [0.003]*4+[0.004]*4 #down/up
                VBFH_pdf[key] = [0.021]*4
            elif 'ggZH' in key:
                ggZH_higgsPt[key] = [0.194, 0.198, 0.208, 0.208, 0.120, 0.123, 0.134, 0.133] #down/up,ggZH
                ggZH_qcdScale[key] = [0.251]*4+[0.189]*4 #down/up
                ggZH_pdf[key] = [0.024]*4
#                 scaling[key] = [0.2]*4
            elif 'ZH' in key: 
                ZH_higgsPt[key] = [0.125, 0.057, 0.01, 0.017, 0.033,0.018, 0.006, 0.006] #down/up
                ZH_qcdScale[key] = [0.006]*4+[0.005]*4 #down/up
                ZH_pdf[key] = [0.019]*4
            elif 'WH' in key: 
                WH_higgsPt[key] = [0.071,0.040, 0.003, 0.054, 0.026, 0.024, 0.004, 0.018] #down/up
                WH_qcdScale[key] = [0.007]*4+[0.005]*4 #down/up
                WH_pdf[key] = [0.019]*4
            elif 'ttH' in key: 
                ttH_higgsPt[key] = [0.012, 0.148, 0.051, 0.028, 0.007, 0.066, 0.016, 0.010] #down/up
                ttH_qcdScale[key] = [0.092]*4+[0.058]*4 #down/up
                ttH_pdf[key] = [0.036]*4
#             else:#ggZH reweight
# #                 higgsPtWeight[key] = [0.125, 0.159, 0.051, 0.054, 0.033,0.066, 0.016, 0.018] #down/up, for the others mode, scaling
# #                 theory[key] = [0.092]*4+[0.058]*4 #down/up
# #                 pdf[key] = [0.036]*4
# #                 theory[key] = [0.092]*4+[0.058]*4 #down/up
#                   ######ggZH
# #                 higgsPtWeight[key] = [0.194, 0.198, 0.208, 0.208, 0.120, 0.123, 0.134, 0.133] #down/up,ggZH
# #                 theory[key] = [0.251]*4+[0.189]*4 #down/up
# #                 pdf[key] = [0.024]*4
#                 higgsPtWeight[key] = [0.125, 0.057, 0.01, 0.017, 0.033,0.018, 0.006, 0.006] #down/up
#                 theory[key] = [0.006]*4+[0.005]*4 #down/up
#                 pdf[key] = [0.019]*4
#                 scaling[key] = [0.2]*4
            mc_stats[k] = []
            if p == 'ggZH': scaling[key] = [0.2]*4
            else: scaling[key] = [0,0,0,0]


# sig_eff = [lumi, muonVeto, jetVeto,rechitVeto,time, time_spread, cut_based_eff_unc, clusterEff_unc,JES,pileup, higgsPtWeight, theory, pdf,mc_stats,scaling]
# sig_eff_name = ['lumi','muonVeto','jetVeto','rechitVeto','time','time_spread','cut_based_eff_unc', 'clusterEff_unc','JES','pileup', 'higgsPtWeight','theory', 'pdf','mc_stats','scaling']

sig_eff = [lumi, muonVeto, jetVeto,rechitVeto,time, time_spread, cut_based_eff_unc, clusterEff_unc,JES,pileup, \
           ggH_higgsPt, VBFH_higgsPt, WH_higgsPt, ZH_higgsPt, ttH_higgsPt, ggZH_higgsPt,  \
           ggH_qcdScale, VBFH_qcdScale, WH_qcdScale, ZH_qcdScale, ttH_qcdScale, ggZH_qcdScale, \
           ggH_pdf,VBFH_pdf, WH_pdf, ZH_pdf, ttH_pdf, ggZH_pdf,\
           scaling, mc_stats]
sig_eff_name = ['lumi','muonVeto','jetVeto','rechitVeto','time','time_spread','cut_based_eff_unc', 'clusterEff_unc','JES','pileup', \
                'ggH_higgsPt', 'VBFH_higgsPt', 'WH_higgsPt', 'ZH_higgsPt', 'ttH_higgsPt', 'ggZH_higgsPt',\
                'ggH_qcdScale', 'VBFH_qcdScale', 'WH_qcdScale', 'ZH_qcdScale', 'ttH_qcdScale', 'ggZH_qcdScale', \
                'ggH_pdf','VBFH_pdf', 'WH_pdf', 'ZH_pdf', 'ttH_pdf', 'ggZH_pdf',\
                'ggZH_reweight','mc_stats']



simulation = [muonVeto, jetVeto,rechitVeto,time, time_spread, cut_based_eff_unc, clusterEff_unc]
simulation_name = ['muonVeto','jetVeto','rechitVeto','time','time_spread','cut_based_eff_unc', 'clusterEff_unc']

veto = [muonVeto, jetVeto,rechitVeto]
veto_name = ['muonVeto','jetVeto','rechitVeto']
assert(len(sig_eff)==len(sig_eff_name))

In [8]:
signal_correction = 1
for corr in veto:
    signal_correction *= (1-corr[k][0])
    print("signal_correction", signal_correction)
print("signal_correction", signal_correction)

signal_correction 0.9468
signal_correction 0.93221928
signal_correction 0.90332048232
signal_correction 0.90332048232


# Signal Contamination

In [6]:
ctaus = ['5','10','30','40', '50','60','80', '100','200','300','500','1000', '2000','3000','5000', '10000', '20000','30000','50000',\
         '100000', '200000', '300000', '500000', '1000000'] #mm
ctaus = ['100', '500','1000', '5000', '10000',\
         '100000', '1000000'] #mm
ctaus = ['500', '1000', '10000', '100000', '1000000']
BR = 0.002
N_RECHIT_CUT = 130
DPHI_CUT = 0.75      
# DPHI_CUT = 0.6   
jetmet = 0
if jetmet: var = jetMet_dPhiMin30
else:var = dphiMet_cluster
for N_RECHIT_CUT in [170]:
    factor = len(var['data_intime_vr'])/len(var['data_oot_vr'])
    k = 'data_oot_sr'
    bkg = 0
    if jetmet: bkg = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] < DPHI_CUT)) 
    else: bkg = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] >= DPHI_CUT))
    print(bkg)
    bkg *= factor
    for m in mass:
        signal_rate = []
        unc_rate = []
        for ct in ctaus:   
            signal = 0
            shape_unc_temp = 0
            ctf = int(ct)
            for i, y in enumerate(years):
                if ctf < OLD_CTAU[0]:
                    old_ctau_temp = np.array([OLD_CTAU[0]])
                else:
                    for j, ct0 in enumerate(OLD_CTAU):
                        if ct0 == ctf: 
                            old_ctau_temp = np.array([ctf])
                            break
                        elif ct0 > ctf:
                            old_ctau_temp = np.array([OLD_CTAU[j-1], OLD_CTAU[j]])
                            old_ctau_temp = np.array([OLD_CTAU[j]])
                            break
                        if j == len(OLD_CTAU)-1: 
                            old_ctau_temp = np.array([OLD_CTAU[j]])
                weight_sum = 0
                weight_len = 0
                for j,ct0 in enumerate(old_ctau_temp):
                    if VBFH: prods = ['ggH', 'VBFH']
                    else: prods = ['ggH']
                    for prod in prods:
                        k = 'MC_'+y+'_'+prod+'_'+str(m)+'_'+str(ct0)
                        T = tree[k]
                        if np.count_nonzero(sel_ev[k]) == 0: continue
                        gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                        if len(gLLP_ctau) == 0: continue
                        weight_ctau = weight_calc(gLLP_ctau, float(ct)/10, old_ctau_temp/10) # convert everything to cm
                        w = pileupWeight[k] * weight[k] * weight_ctau * higgsPtWeight[k]
                        shape_unc = signal_weight(unc_list, np.abs(cscRechitClusterEta[k][bdt_sel[k]]))
                        # higgs pt, signal uncertainty weight
#                         cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]<DPHI_CUT)
                        if jetmet: cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]<DPHI_CUT) #bin D
                        else: cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]>=DPHI_CUT) #bin A
                        signal += np.sum(w[cond])
                        shape_unc_temp = (shape_unc_temp**2+np.sum(w[cond]*shape_unc[cond])**2)**0.5
    #                     print(np.sum(w[cond]*sf_facScaleUp[k][cond])/np.sum(w[cond]), np.sum(w[cond]*sf_facScaleDown[k][cond])/np.sum(w[cond]))
    #                     print(np.sum(w[cond]*sf_renScaleUp[k][cond])/np.sum(w[cond]), np.sum(w[cond]*sf_renScaleDown[k][cond])/np.sum(w[cond]))
    #                     print(np.sum(w[cond]*sf_facRenScaleUp[k][cond])/np.sum(w[cond]), np.sum(w[cond]*sf_facRenScaleDown[k][cond])/np.sum(w[cond]))


            signal_rate.append(signal)
            unc_rate.append(shape_unc_temp)
        signal_rate = np.array(signal_rate)
        unc_rate = np.array(unc_rate)
#         print(N_RECHIT_CUT, '\t', signal_rate[0]*BR)
#         print(m, '\t', '\t'.join(map(str, [round(num,2) for num in signal_rate*BR])))
        print(m, '\t', '\t'.join(map(str, [round(num/bkg,2) for num in signal_rate*BR])))


#         print(m, '\t'.join(map(str,[round(num,2) for num in unc_rate*BR]))) #signal mc relative uncertainty


#     print(m, '\t'.join(map(str,unc_rate/signal_rate))) #signal mc relative uncertainty

print(bkg)


1


NameError: name 'VBFH' is not defined

# make datacard cleaned version

In [43]:
import os
import math
import importlib
importlib.reload(sys.modules['helper'])
from helper import make_datacard, make_datacard_2sig, weight_calc
#methodB use oot vr and ABCD method
#methodC use the fit functions
N_RECHIT_CUTS = np.arange(60,220, 10)
# N_RECHIT_CUTS = [130]
# ctaus = ['5','10','30','40', '50','60','80', '100','200','300','500','1000', '2000','3000','5000', '10000', '20000','30000','50000',\
#          '100000', '200000', '300000', '500000', '1000000'] #mm
ctaus = [ '20', '25', '200000', '300000', '500000', '1000000', '2000000', '3000000', '5000000', '10000000'] #mm

# ctaus = ['1000', '10000', '100000']
UNBLIND = 2
# 0 use OOT region, 1 unblind ABC, 2 unblind ABCD
method = 2
fit_function = 'se' #se, me, sp
jetmet=0
datacard_version  = 'v11'
# v9 split the 3 theory uncertainties for each production mode
if datacard_version == 'v1':
    bkg_unc = 1.0
    sig_unc0 = 0.2
elif datacard_version == 'v2':
    bkg_unc = 0.5
    sig_unc0 = 0.2
elif datacard_version == 'v3':
    bkg_unc = 1.0
    sig_unc0 = 0.3
elif datacard_version == 'v4':#used non-closure+stat error for background
    bkg_unc = 0.0
    sig_unc0 = 0.2
elif datacard_version == 'v5':#used stat error for background
    bkg_unc = 0.0
    sig_unc0 = 0.2
elif datacard_version == 'v11':
    bkg_unc = [] #ABCD closure, sum of stats from two VR
    bkg_unc_name = ['ABCD']
else:
    bkg_unc = [] #list of two uncertainty, each element is N for the gamma uncertainty distriubtion, alpha is calculated in the makr_datacard method
    bkg_unc_name = ['ABCD_intimeVR', 'ABCD_OOTSR']
    print("ERROR in uncertainty assignment")
if jetmet: 
    var = jetMet_dPhiMin30
    DPHI_CUT = 0.6
    DPHI_CUTS = [0.6]
else: 
    var = dphiMet_cluster
    DPHI_CUT = 0.75
    DPHI_CUTS = [0.75]

# print(bdtBkgEff_nrechit.shape)
bkg = []
print(N_RECHIT_CUTS)
for N_RECHIT_CUT in N_RECHIT_CUTS:
    if not N_RECHIT_CUT==130:continue
    for DPHI_CUT in DPHI_CUTS:
        bkg_unc = []
        ##### method B (use OOT high BDT region and tha ratio between intime and OOT)#####
        k = 'data_intime_vr'
        a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] >= DPHI_CUT))
        b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] >= DPHI_CUT)) 
        c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] < DPHI_CUT)) 
        d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] < DPHI_CUT)) 
        pred = a*c/b
        if datacard_version == 'v4': 
            unc = (abs(d-pred)/pred)**2+pred/pred**2
        elif datacard_version == 'v11':
            bkg_unc.append(pred)
        else:
            bkg_unc.append(1./pred**0.5) # only the statistical uncertainty(relative)
        factor = len(var['data_intime_vr'])/len(var['data_oot_vr'])
        k = 'data_oot_sr'
        a = factor * np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] >= DPHI_CUT))
        b = factor * np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] >= DPHI_CUT)) 
        c = factor * np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] < DPHI_CUT)) 
        d = factor * np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] < DPHI_CUT)) 

        pred = a*c/b
        if datacard_version == 'v4': 
            bkg_unc = (unc + (abs(d-pred)/pred)**2+pred/pred**2)**0.5
        elif datacard_version == 'v11':
            
            bkg_unc[0] += pred/factor # only the statistical uncertainty(relative)
            bkg_unc[0] = 1./(bkg_unc[0]**0.5)
        else:
            bkg_unc.append(1./(pred/factor)**0.5) # only the statistical uncertainty(relative)
        bkg_rate = np.array([a,b,c,pred])
        observation = bkg_rate
        if UNBLIND > 0 :
            k = 'data_intime_sr'
            a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] >= DPHI_CUT))
            b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] >= DPHI_CUT)) 
            c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] < DPHI_CUT)) 
            d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] < DPHI_CUT)) 
            bkg_rate = np.array([a,b,c,a*c/b])
            if UNBLIND == 1:observation= np.array([a,b,c,a*c/b])
            else: observation= np.array([a,b,c,d])
        print(N_RECHIT_CUT, observation, bkg_unc)
        
        #####
        # a = c*c1*c2
        # b = c1* c
        # c = c
        # d = c2*c
        # start adding signal
        #####
        outDataCardsDir = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards/'+\
        ntupler_version+analyzer_version+'/'
        if cut_based:outDataCardsDir += 'cut_based_'+cut_based_version+'/'
        else: outDataCardsDir += bdt_name+'/'
        outDataCardsDir += datacard_version+'/methodB/'
        if UNBLIND == 0: outDataCardsDir += 'blind/'
        elif UNBLIND == 1: outDataCardsDir += 'unblindABC/'
        else:outDataCardsDir += 'unblind/'
        if not os.path.isdir(outDataCardsDir):os.makedirs(outDataCardsDir)
        print(outDataCardsDir)
        sig_norm = []
        for m in mass:
            for ct in ctaus:
                dphi = str(DPHI_CUT).replace(".", "p")
                if len(prod) == 5: modelName = 'allProd_HToSSTo'+decay
                elif len(prod) ==6:  modelName = 'allProd_withggZH_HToSSTo'+decay
                else: modelName = ('_').join(prod)+'_HToSSTo'+decay
                modelName = modelName + '_mh125_mx'+str(m)+'_ctau'+str(ct)+'mm_nRechit'+str(N_RECHIT_CUT)+'dPhiCluster'+dphi
                
                signal_rate = {}
                mc_stat_unc = {}
                gmn = {}
                sig_unc = {}
                ctf = int(ct)
                ct_list = 10**int(math.log10(ctf))
                if ctf < OLD_CTAU[0]: ct_list = [OLD_CTAU[0]]
                elif ctf>OLD_CTAU[-1]: ct_list = [OLD_CTAU[-1]]
                elif ct_list == int(ct): ct_list = [int(ct)]
                else:ct_list = [ct_list,ct_list*10]
                for p in prod:                    
                    signal_rate[p] = np.zeros((4,))
                    mc_stat_unc[p] = np.zeros((4,))
                    gmn[p] = np.zeros((4,))
                    sig_unc[p] = []
                    if p == 'others':
                        for i in range(4):
                            signal_rate[p][i] = signal_rate['ggH'][i]*0.52
                            mc_stat_unc[p][i] = (mc_stat_unc['ggH'][i]*signal_rate[p][i])**2
                    else:
                        for i, ct0 in enumerate(ct_list):
                            k = 'MC_'+p+'_'+str(m)+'_'+str(ct0)
                            T = tree[k]
                            if np.count_nonzero(sel_ev[k])==0:continue
#                             gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                            gLLP_ctau = T.array('gLLP_ctau')[sel_ev[k]][bdt_sel[k]]

                            weight_ctau = weight_calc(gLLP_ctau, int(ct)/10, int(ct0)/10) # convert everything to cm
                            gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                            if len(ct_list) == 1:weight_cond = gLLP_ctau >= 0
                            else:
                                if i == 0 : weight_cond = gLLP_ctau<int(ct_list[0]/2)
                                else: weight_cond = gLLP_ctau>=int(ct_list[0]/2)
                            w = weight[k]*weight_ctau
                            cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]>=DPHI_CUT), weight_cond)
                            signal_rate[p][0]+=np.sum(w[cond])
                            mc_stat_unc[p][0]+=np.sum(w[cond]**2)
                            gmn[p][0] += len(w[cond])
                            cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]>=DPHI_CUT), weight_cond)
                            signal_rate[p][1]+=np.sum(w[cond])
                            mc_stat_unc[p][1]+=np.sum(w[cond]**2)
                            gmn[p][1] += len(w[cond])
                            cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]<DPHI_CUT), weight_cond)
                            signal_rate[p][2]+=np.sum(w[cond])
                            mc_stat_unc[p][2]+=np.sum(w[cond]**2)
                            gmn[p][2] += len(w[cond])
                            cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]<DPHI_CUT), weight_cond)
                            signal_rate[p][3]+=np.sum(w[cond])
                            mc_stat_unc[p][3]+=np.sum(w[cond]**2)
                            gmn[p][3] += len(w[cond])
                            # apply the corrections
                            for j in [0,1,2,3]:
                                if signal_rate[p][j]<0.0:
                                    signal_rate[p][j]=0.0
                                    mc_stat_unc[p][j]=0.0
                                    gmn[p][j] = 0.0
                                else:
                                    signal_correction = 1
                                    for corr in veto:
                                        signal_correction *= (1-corr[k][j])
                                    signal_rate[p][j] *= signal_correction

#                     mc_stat_unc[p] = list(np.nan_to_num(np.sqrt(mc_stat_unc[p])/signal_rate[p])) #convert absolute to relative uncertainty
                    
                    mc_stat_unc[p] = np.nan_to_num(signal_rate[p]/np.sqrt(mc_stat_unc[p]))**2 #gmn
                    mc_stat_unc[p][mc_stat_unc[p] == np.inf] = 0.0
                    
                    mc_stat_unc[p] = list(mc_stat_unc[p])
                    mc_stat_unc[p] = list(gmn[p])
                    if p == 'others': k = k.replace("ggH", "others")
                    for j, ele in enumerate(sig_eff):# go through each uncertainty
                        if j == len(sig_eff)-1: # if mc_stats
                            sig_unc[p].append(mc_stat_unc[p])
#                         elif sig_eff_name[j] in simulation_name:
# #                             print(type(ele[k]))
#                             sig_unc[p].append(np.array(ele[k])*0.5)
                        else:
                            sig_unc[p].append(ele[k])
                   
                
                norm = np.sum(signal_rate['ggH'])
                if norm == 0.0:continue
                sig_norm.append(norm)

                for k,v in signal_rate.items():signal_rate[k] = v/norm
                print(modelName, norm, signal_rate['ggH'], mc_stat_unc['ggH'])
#                 make_datacard_2sig(outDataCardsDir, modelName, signal_rate, norm, bkg_rate, observation, \
#                                   bkg_unc, bkg_unc_name, sig_unc, sig_eff_name)
                

[ 60  70  80  90 100 110 120 130 140 150 160 170 180 190 200 210]
130 [ 3 96 47  3] [0.6113341128409626]
/storage/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards/V1p17/v2/v106//cut_based_v4/v11/methodB/unblind/
allProd_withggZH_HToSSTodddd_mh125_mx7_ctau20mm_nRechit130dPhiCluster0p75 98.23446507816516 [1.17623011e-07 1.89605756e-04 2.48804169e-02 9.74929860e-01] [12.0, 5.0, 189.0, 1629.0]
allProd_withggZH_HToSSTodddd_mh125_mx7_ctau25mm_nRechit130dPhiCluster0p75 194.20329920580843 [1.90345319e-06 3.56654654e-04 3.57575976e-02 9.63883844e-01] [12.0, 5.0, 189.0, 1629.0]
allProd_withggZH_HToSSTodddd_mh125_mx7_ctau200000mm_nRechit130dPhiCluster0p75 28.689607794280327 [0.22518319 0.18718435 0.15867577 0.42895668] [12.0, 10.0, 8.0, 24.0]
allProd_withggZH_HToSSTodddd_mh125_mx7_ctau300000mm_nRechit130dPhiCluster0p75 19.408181431590176 [0.22567654 0.18844127 0.15975726 0.42612493] [12.0, 10.0, 8.0, 24.0]
allProd_withggZH_HToSSTodddd_mh125_mx7_

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:192: RuntimeWarning: invalid value encountered in true_divide


allProd_withggZH_HToSSTodddd_mh125_mx15_ctau200000mm_nRechit130dPhiCluster0p75 68.75490472931943 [0.23658882 0.18301615 0.12738014 0.45301489] [31.0, 29.0, 24.0, 62.0]
allProd_withggZH_HToSSTodddd_mh125_mx15_ctau300000mm_nRechit130dPhiCluster0p75 46.281672814636806 [0.23769695 0.18461725 0.12487678 0.45280902] [31.0, 29.0, 24.0, 62.0]
allProd_withggZH_HToSSTodddd_mh125_mx15_ctau500000mm_nRechit130dPhiCluster0p75 27.986343356022306 [0.23858339 0.18590679 0.1228619  0.45264792] [31.0, 29.0, 24.0, 62.0]
allProd_withggZH_HToSSTodddd_mh125_mx15_ctau1000000mm_nRechit130dPhiCluster0p75 14.075774434365504 [0.23924815 0.18687898 0.12134376 0.45252911] [31.0, 29.0, 24.0, 62.0]
allProd_withggZH_HToSSTodddd_mh125_mx15_ctau2000000mm_nRechit130dPhiCluster0p75 7.058716996785643 [0.2395805  0.1873667  0.12058248 0.45247033] [31.0, 29.0, 24.0, 62.0]
allProd_withggZH_HToSSTodddd_mh125_mx15_ctau3000000mm_nRechit130dPhiCluster0p75 4.710457990785787 [0.23969127 0.18752951 0.1203284  0.45245082] [31.0, 29.0

# unblind

In [9]:
import math

In [33]:
k = 'data_intime_sr'

n_ev = 5000
var = np.abs(dphiMet_cluster[k])
DPHI_CUT = 0.75
print("Nrechits, A, B, C, pred, D")
for N_RECHIT_CUT in np.arange(60,240,10):
    a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var>=DPHI_CUT))
    b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var>=DPHI_CUT))
    c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var<DPHI_CUT))
    d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var<DPHI_CUT))
    pred = c/b*a
    unc_pred = (1./c + 1./b + 1./a)**0.5*(c/b*a)
#     if math.isnan(unc_pred): z_value = float("nan")
#     else:
#         mu = np.random.normal(pred, unc_pred, n_ev)
#         p_value = 0.0
#         for i in mu:
#             if i < 0:continue
#             n = np.random.poisson(i, n_ev)
#             p_value += np.count_nonzero(n>=d)

#         p_value = p_value/n_ev**2
#         z_value = abs(norm.ppf(p_value))
    print(N_RECHIT_CUT, '\t',a,'\t',b,'\t',c,'\t',d,'\t', round(c/b*a, 2), '\t',\
          round( (1./c + 1./b + 1./a)**0.5*(c/b*a), 2))
#     print(N_RECHIT_CUT, '\t',a,'\t',b,'\t',c,'\t',d,'\t', round(c/b*a, 2), '\t',\
#           round( (1./c + 1./b + 1./a)**0.5*(c/b*a), 2), '\t', round(z_value,2))


Nrechits, A, B, C, pred, D
60 	 47 	 52 	 20 	 29 	 18.08 	 5.44
70 	 22 	 77 	 32 	 17 	 9.14 	 2.74
80 	 11 	 88 	 38 	 11 	 4.75 	 1.7
90 	 9 	 90 	 42 	 7 	 4.2 	 1.6
100 	 3 	 96 	 42 	 7 	 1.31 	 0.8
110 	 3 	 96 	 42 	 7 	 1.31 	 0.8
120 	 3 	 96 	 43 	 6 	 1.34 	 0.81
130 	 3 	 96 	 47 	 2 	 1.47 	 0.89
140 	 2 	 97 	 47 	 2 	 0.97 	 0.71
150 	 2 	 97 	 47 	 2 	 0.97 	 0.71
160 	 2 	 97 	 48 	 1 	 0.99 	 0.72
170 	 1 	 98 	 48 	 1 	 0.49 	 0.5


ZeroDivisionError: float division by zero

In [39]:
for k, v in dphiMet_cluster.items():
    N_RECHIT_CUT = 130
    DPHI_CUT = 0.75
    var=v
    a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var>=DPHI_CUT))
    b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var>=DPHI_CUT))
    c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var<DPHI_CUT))
    d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var<DPHI_CUT))
    if not 'data' in k:continue
    print(k, len(v), a,b,c,d)

data 149 3 96 47 3
data_oot_sr 477 5 373 97 2
data_oot_vr 986 26 732 222 6
data_intime_vr 350 5 269 74 2
data_intime_sr 149 3 96 47 3


In [ ]:
data 148 3 96 47 2
data_oot_sr 477 5 373 97 2
data_oot_vr 986 26 732 222 6
data_intime_vr 350 5 269 74 2
data_intime_sr 148 3 96 47 2

In [ ]:
k = 'data_intime_sr'
N_RECHIT_CUT = 130
var = np.abs(dphiMet_cluster[k])
DPHI_CUT = 0.75
cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var<DPHI_CUT)
for i in range(np.count_nonzero(cond)):
    print(str(runNum[k][cond][i])+":"+str(lumiSec[k][cond][i])+":"+str(evtNum[k][cond][i]))
    print(cscRechitClusterEta[k][bdt_sel[k]][cond][i], cscRechitClusterPhi[k][bdt_sel[k]][cond][i],metPhi[k][cond][i],var[cond][i],met[k][cond][i], nCsc_JetMuonVetoCluster0p4_Me1112Veto[k][cond][i])

In [17]:
len(cscRechitClusterEta[k][bdt_sel[k]])

149

In [29]:
met[k]

KeyError: 'data_intime_sr'